# Trabalho Prático 1 - Introdução à Computação Visual

## Introdução

O objetivo deste trabalho é a implementação de um método de compressão de imagens em escala de cinza. Deverá ser possível comprimir uma imagem sem compressão passada como entrada, salvar essa imagem e, posteriormente, passar como entrada uma imagem comprimida pelo método aqui implementado e exibi-la. A compressão pode ser tanto com perda quanto sem perda. O PSNR deverá ser exibido após a compressão da imagem.

## Implementação

O método implementado neste trabalho consiste na implementação da codificação preditiva sem perdas com o processo da codificação de Huffman no final. Para uma dada imagem inserida, o compressor irá salvar somente o primeiro pixel da imagem, presente na origem (0, 0). A partir daí, uma função de predição irá tentar retornar o valor do próximo pixel. A diferença entre o valor predito e o valor real será o erro obtido. Esse erro será o valor salvo para codificação naquele ponto, de forma a reduzir a entropia da imagem, obtendo, no final, uma matriz com uma frequência alta de poucos valores. Os valores dessa matriz serão, ao final, codificados em uma árvore com a codificação de Huffman, diminuindo o número médio de bits por pixel.

Para obter a imagem novamente, será necessário somente realizar um processo para reverter a codificação de Huffman e, a partir do primeiro pixel, gerar os pixels seguintes, de acordo com a diferença obtida entre a função inversa de predição e o valor real, que estará na matriz.

Para podermos testar o impacto de cada um dos dois métodos utilizados na taxa de compressão, o algoritmo será modularizado e parametrizado de forma que possa ser utilizado livremente qualquer um dos dois métodos a qualquer momento, além de poder utilizar os dois em conjunto.

### Importações

In [1]:
%matplotlib inline
import cv2
import numpy as np
from matplotlib import pyplot as plt
import heapq

### Definições de variáveis constantes

In [2]:
INPUT_UNCOMPRESSED_IMG_PATH = "lena512.pgm"
COMPRESSED_IMG_PATH = "lena512.mycompress"

# Alterar essas variáveis irá ativar ou desativar esses métodos de compressão.
USE_PREDICTION = True
USE_HUFFMAN = True

### Implementação da codificação preditiva e sua decodificação

In [3]:
def predictionFunction(x, raveledImg):
    return raveledImg[x - 1] - 256

def predictiveCode(img):
    imgShape = img.shape
    raveledImg = img.ravel()
    firstElement = raveledImg[0]
    errorMatrix = np.empty(raveledImg.shape, dtype=int)
    # O primeiro elemento será exatamente o primeiro valor.
    errorMatrix[0] = raveledImg[0]
    for x, currentValue in enumerate(raveledImg[1:], 1):
        predictedValue = predictionFunction(x, raveledImg)
        error = int(currentValue) - int(predictedValue)
        errorMatrix[x] = error
    errorMatrix = errorMatrix.reshape(imgShape)
    return errorMatrix
    
def predictiveDecode(errorMatrix):
    imgShape = errorMatrix.shape
    errorMatrix = errorMatrix.ravel()
    img = np.empty(errorMatrix.shape, dtype=int)
    # Como a função de predição utiliza o elemento anterior, ao menos o primeiro elemento deve estar preenchido.
    img[0] = errorMatrix[0]
    for x, error in enumerate(errorMatrix[1:], 1):
        predictedValue = predictionFunction(x, img)
        img[x] = predictedValue + error
    img = img.reshape(imgShape)
    return img

### Implementação da codificação de Huffman e sua decodificação

In [4]:
class Node:
    def __init__(self, left, right):
        self.left = left
        self.right = right
    
    # Sobrescreve operadores de comparação que são usados na fila de prioridade.
    # Como essa comparação só é feita quando a frequência é igual para dois valores, a ordem não fará diferença.
    def __lt__(self, other):
        return True

    def __gt__(self, other):
        return True

def generatePriorityQueue(img):
    values, frequencies = np.unique(img, return_counts=True)
    priorityQueue = np.asarray((frequencies, values)).T.tolist()
    heapq.heapify(priorityQueue)
    return priorityQueue

def generateBinaryTree(priorityQueue):
    for _ in range(len(priorityQueue) - 1):
        lowerElement = heapq.heappop(priorityQueue)
        secondLowerElement = heapq.heappop(priorityQueue)
        # O primeiro valor da tupla é a frequência e o segundo é o valor em si.
        newFrequency = lowerElement[0] + secondLowerElement[0]
        node = Node(lowerElement[1], secondLowerElement[1])
        heapq.heappush(priorityQueue, [newFrequency, node])
    return heapq.heappop(priorityQueue)[1]

def recursiveDFS(currentNode, code, currentValue):
    currentValue = currentValue + '0'
    if isinstance(currentNode.left, Node):
        recursiveDFS(currentNode.left, code, currentValue)
    else:
        code[currentNode.left] = currentValue
        
    currentValue = currentValue[:-1] + '1'
    if isinstance(currentNode.right, Node):
        recursiveDFS(currentNode.right, code, currentValue)
    else:
        code[currentNode.right] = currentValue

def generateCode(binaryTree):
    initialValue = ''
    code = {}
    recursiveDFS(binaryTree, code, initialValue)
    return code
        
def huffmanCode(img):
    priorityQueue = generatePriorityQueue(img)
    binaryTree = generateBinaryTree(priorityQueue)
    code = generateCode(binaryTree)
    return code

### Implementação da codificação binária tradicional

In [5]:
def intToBinary(number, length):
    return bin(number)[2:].zfill(length)

def binaryCode(img):
    code = {}
    uniqueValues = np.unique(img)
    minimumBitsLength = len(uniqueValues).bit_length()
    for i, value in enumerate(uniqueValues):
        code[value] = intToBinary(i, minimumBitsLength)
    return code

### Definição das funções de escrita e leitura de imagens comprimidas

In [6]:
def encodeHeader(img, code):
    header = '1' if USE_PREDICTION else '0'
    header += '1' if USE_HUFFMAN else '0'
    # Armazena as dimensões da imagem usando 2 bytes
    header += intToBinary(len(img), 16)
    header += intToBinary(len(img[0]), 16)
    for key, value in code.items():
        # O intervalo de valores possíveis é [0, 511].
        # Esses valores podem ser representados usando, no máximo, 9 bits
        header += intToBinary(key, 9)
        # O intervalo de valores possíveis é [0, 9] (tamanho da string acima).
        # Esses valores podem ser representados usando, no máximo, 4 bits
        header += intToBinary(len(value), 4)
        header += value
    return header

def encodeBody(img, code):
    body = ''
    for line in img:
        for cell in line:
            body += code[cell]
    return body

def generateCompressedImageByteArray(header, body):
    compressedImage = header + body
    lengthDifferenceToEight = len(compressedImage) % 8
    if lengthDifferenceToEight != 0:
        compressedImage += intToBinary(0, 8 - lengthDifferenceToEight)
    return bytearray(int(compressedImage[i:i+8], 2) for i in range(0, len(compressedImage), 8))    

def writeCompressedImage(img, code):
    header = encodeHeader(img, code)
    body = encodeBody(img, code)
    imageByteArray = generateCompressedImageByteArray(header, body)
    with open(COMPRESSED_IMG_PATH, 'wb') as file:
        file.write(imageByteArray)
    
def readCompressedImage(file):
    None

### Definição das funções gerais

In [7]:
def showImage(img, title="Image"):
    plt.title(title)
    plt.imshow(img, cmap = 'gray')
    plt.show()

def compressImageAndShowResults():
    img = cv2.imread(INPUT_UNCOMPRESSED_IMG_PATH, cv2.IMREAD_GRAYSCALE)
    if USE_PREDICTION:
        img = predictiveCode(img)
    if USE_HUFFMAN:
        code = huffmanCode(img)
    else:
        code = binaryCode(img)
    writeCompressedImage(img, code)

### Sandbox para uso das funções

In [8]:
USE_PREDICTION = True
USE_HUFFMAN = True
compressImageAndShowResults()

In [9]:
USE_PREDICTION = True
USE_HUFFMAN = False
compressImageAndShowResults()

In [10]:
USE_PREDICTION = False
USE_HUFFMAN = True
compressImageAndShowResults()

In [11]:
# A imagem "comprimida" gerada neste caso sempre terá um tamanho
# quase idêntico à imagem original, com a diferença somente do
# acréscimo do tamanho do cabeçalho.
USE_PREDICTION = False
USE_HUFFMAN = False
compressImageAndShowResults()